In [35]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import plotly.express as px

%matplotlib inline
from sklearn import svm

In [36]:
df = pd.read_csv("top.csv")
df.head()
#df.info()

,Unnamed: 0,genre,name,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,key,tempo
0,0,classical,Handel / Orch. Hale: Keyboard Suite in D Minor...,George Frideric Handel,93,33,-24,60,927,830,95,51,2,67
1,1,classical,"Goldberg Variations, BWV 988: Aria",Johann Sebastian Bach,454,13,-29,51,995,943,73,244,4,130
2,2,classical,"Clair de Lune, L. 32",Claude Debussy,335,5,-31,37,994,912,62,39,1,65
3,3,classical,"Sonata No. 14 ""Moonlight"" in C-Sharp Minor"", O...",Ludwig van Beethoven,184,5,-37,43,995,887,173,151,1,170
4,4,classical,Miroirs: III. Une barque sur l'océan,Maurice Ravel,170,47,-27,43,981,906,79,30,2,75


In [37]:
#extracting columns we will use to base classifications
#dropping unnecessary columns

# UNCOMMENT FOR MEGADATASET
# df.drop(columns=['year', 'release_date', 'id', 'duration_ms', 'artists', 
#                  'name', 'explicit', 'popularity', 'mode', 'key', 'loudness'], inplace=True)

# df.drop(columns=['Unnamed: 0','year','dur','pop'], inplace= True)
# df

df.drop(columns=['Unnamed: 0'], inplace=True)
df

,genre,name,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,key,tempo
0,classical,Handel / Orch. Hale: Keyboard Suite in D Minor...,George Frideric Handel,93,33,-24,60,927,830,95,51,2,67
1,classical,"Goldberg Variations, BWV 988: Aria",Johann Sebastian Bach,454,13,-29,51,995,943,73,244,4,130
2,classical,"Clair de Lune, L. 32",Claude Debussy,335,5,-31,37,994,912,62,39,1,65
3,classical,"Sonata No. 14 ""Moonlight"" in C-Sharp Minor"", O...",Ludwig van Beethoven,184,5,-37,43,995,887,173,151,1,170
4,classical,Miroirs: III. Une barque sur l'océan,Maurice Ravel,170,47,-27,43,981,906,79,30,2,75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,rock,Shine,Collective Soul,540,423,-8,28,294,88,152,353,6,150
996,rock,Helplessly Hoping - 2005 Remaster,"Crosby, Stills & Nash",567,159,-13,33,914,0,111,436,7,146
997,rock,Everlong,Foo Fighters,413,881,-5,36,0,0,80,364,11,158
998,rock,The Power Of Love,Huey Lewis & The News,758,831,-5,32,85,0,101,958,5,118


In [39]:
# fig = px.scatter(df, x='bpm', y='nrgy',color='top genre', hover_name='artist',hover_data=['title'])
fig = px.scatter(df, x='tempo', y='energy',color='genre', hover_name='name', hover_data=['artist'])
fig.show()

In [40]:
k = 5

data = np.array_split(df, k) # split the data into 5 roughly equal groups of samples

results = [] # accuracy

# create the support vector machines with the linear, poly, and rbf kernels
clfs = [svm.SVC(kernel="linear"), svm.SVC(kernel="poly"), svm.SVC(kernel="rbf")]

for i in range(k):                                          # run 5 experiments for 5-fold cross-validation
    test_data_set = data[i]                                 # uses one group of data as the test data
    train_data_set = pd.concat(data[:i] + data[i+1:])       # train data takes the rest

    test_labels = np.array(test_data_set.iloc[:, 2])        # use the first column as the label
    test_data = np.array(test_data_set.iloc[:, 3:])         # use the rest for the data

    train_labels = np.array(train_data_set.iloc[:, 2])      # use the first column as the label
    train_data = np.array(train_data_set.iloc[:, 3:])       # use the rest for the data

    experiment_result = []      # the results using each svm for this partitioned data set

    for j in range(len(clfs)):  # use the data set to train each svm
        clfs[j].fit(train_data, train_labels)       # train the svm
        predictions = clfs[j].predict(test_data)    # predict
        accuracy = sum(predictions == test_labels) / len(test_labels)   # get accuracy
        experiment_result.append(accuracy)          # append to the experiment results

    results.append(experiment_result)   # append the row of data to the results

# put the results into a dataframe
results = pd.DataFrame(results)
results.columns = ["Linear", "Poly", "RBF"]

# calculate the average accuracy for each column
avg_accuracy = [sum(results.iloc[:, i]) / len(results) for i in range(3)]
results.loc["Avg"] = avg_accuracy

# display results
print(results)

     Linear   Poly  RBF
0     0.000  0.000  0.0
1     0.010  0.010  0.0
2     0.030  0.030  0.0
3     0.010  0.010  0.0
4     0.030  0.030  0.0
Avg   0.016  0.016  0.0
